# Aggregated & Preprocessed Data — EDA Notebook

**Author:** Virendra

## Overview
This notebook performs exploratory data analysis (EDA), data cleaning, preprocessing, aggregation, and feature engineering on the Uber traffic and weather datasets. The notebook is written in a **very detailed** style (Option A) with explanatory markdown cells for each step to help reviewers understand the workflow.

## 1. Environment & Libraries

Import required Python libraries. If you don't have any of these installed, run `pip install` or use conda.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Display settings for pandas
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

print('Python:', sys.version.splitlines()[0])
print('Pandas:', pd.__version__)
print('NumPy:', np.__version__)

## 2. Load Raw Data

Load the traffic and merged datasets from the specified file paths. If the merged file exists, we'll load it; otherwise we'll load the raw traffic file and fetch/merge weather later.

In [ ]:
# Paths (from user)
uber_path = r"C:\Users\dell8\Downloads\Uber Project\Dataset_Uber Traffic.csv"
merged_path = r"C:\Desktop\Upgrad_Python\Uber Project\merged_uber_weather_lag.csv"

print('Uber path ->', uber_path)
print('Merged path ->', merged_path)

# Load files (use merged if present)
if os.path.exists(merged_path):
    print('Loading merged dataset...')
    df = pd.read_csv(merged_path)
    used_merged = True
else:
    print('Merged not found. Loading raw traffic dataset...')
    traffic = pd.read_csv(uber_path)
    used_merged = False
    df = traffic.copy()

print('\nDataframe shape:', df.shape)
print('\nColumns:')
print(df.columns.tolist())

# Show top rows
df.head()

## 3. Initial Inspection

Check datatypes, missing values, and a quick summary. This helps identify issues early.

In [ ]:
# Basic info
print(df.info())

# Summary stats for numeric columns
print('\nNumeric summary:')
print(df.describe(include='all').transpose().round(3))

# Missing values
print('\nMissing values count:')
print(df.isnull().sum())

## 4. DateTime parsing and correction

Ensure the DateTime column is parsed properly and set correct dtype. We also create time-based features: hour, day_of_week, month, is_weekend.

In [ ]:
if 'DateTime' in df.columns:
    df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True, errors='coerce')
else:
    # try some common alternatives
    for c in df.columns:
        if 'date' in c.lower() or 'time' in c.lower():
            df.rename(columns={c: 'DateTime'}, inplace=True)
            df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True, errors='coerce')
            break

print(df['DateTime'].head())

# Time features
if 'DateTime' in df.columns:
    df['hour'] = df['DateTime'].dt.hour
    df['day_of_week'] = df['DateTime'].dt.dayofweek
    df['month'] = df['DateTime'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)

print(df[['DateTime','hour','day_of_week','month','is_weekend']].head())

## 5. Missing Values Strategy

- Show columns with missing values
- Apply forward/backward fill for weather features if present
- For traffic `Vehicles` consider removing or imputing (we'll forward/backward fill as a safe approach)


In [ ]:
missing = df.isnull().sum()
missing = missing[missing>0].sort_values(ascending=False)
missing

# Example imputation strategy
weather_cols = [c for c in df.columns if c.lower() in ['temp','rhum','wspd','prcp','precip','temperature']]
print('Detected weather-like columns:', weather_cols)

# Fill strategy
if weather_cols:
    df[weather_cols] = df[weather_cols].ffill().bfill()

if 'Vehicles' in df.columns:
    df['Vehicles'] = df['Vehicles'].ffill().bfill()

# Drop rows where DateTime is missing (cannot work without timestamp)
df = df.dropna(subset=['DateTime'])

print('\nMissing values after imputation:')
print(df.isnull().sum())

# Keep a copy of preprocessed dataframe
df_preprocessed = df.copy()

print('Done imputation and cleanup. Shape ->', df.shape)

## 6. Remove Duplicates & Correct Data Types

- Remove exact duplicates
- Ensure numeric columns are numeric


In [ ]:
# Remove duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f'Removed {before-after} duplicate rows')

# Convert numeric columns
num_cols = ['Vehicles','temp','rhum','wspd','prcp']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('Types after numeric conversion:')
print(df.dtypes[df.dtypes!='object'])

## 7. Aggregation: Hourly traffic per Junction

If traffic is already hourly aggregated this will keep it; otherwise we aggregate (sum) Vehicles per hour per junction.

In [ ]:
# Ensure DateTime is rounded/truncated to hour
if 'DateTime' in df.columns:
    df['DateTime_hour'] = df['DateTime'].dt.floor('H')

agg_cols = ['DateTime_hour','Junction']
if 'Vehicles' in df.columns:
    agg_df = df.groupby(agg_cols)['Vehicles'].sum().reset_index().rename(columns={'DateTime_hour':'DateTime'})
else:
    agg_df = df.copy()

print('Aggregated shape:', agg_df.shape)
agg_df.head()

## 8. Merge with Weather Data (if available)

If the merged dataset wasn't available earlier, attempt to load a weather file or merged file to attach weather features. If your `merged_uber_weather_lag.csv` exists we will use it as canonical merged dataset.

In [ ]:
# If user provided merged_path earlier and it exists, load it to get weather
if os.path.exists(merged_path):
    print('Loading user provided merged dataset to ensure weather features are present')
    merged_df = pd.read_csv(merged_path)
    # Try to ensure columns align
    if 'DateTime' in merged_df.columns:
        merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'], dayfirst=True, errors='coerce')
    print('Merged dataset shape:', merged_df.shape)
    display_cols = [c for c in merged_df.columns if c in ['DateTime','Junction','Vehicles','temp','rhum','wspd','prcp']]
    print('Sample from merged dataset:')
    print(merged_df[display_cols].head())
else:
    print('No merged dataset found at given path; ensure you have the weather file separately and merge manually.')

## 9. Feature Engineering

Create time-based features, lag features, rolling-window statistics, and event/weekend flags.

In [ ]:
# Start with agg_df (hourly traffic)
if 'agg_df' in globals():
    df_features = agg_df.copy()
else:
    df_features = df.copy()

# Ensure DateTime
if 'DateTime' in df_features.columns:
    df_features['DateTime'] = pd.to_datetime(df_features['DateTime'], dayfirst=True, errors='coerce')

# Time features
if 'DateTime' in df_features.columns:
    df_features['hour'] = df_features['DateTime'].dt.hour
    df_features['day_of_week'] = df_features['DateTime'].dt.dayofweek
    df_features['month'] = df_features['DateTime'].dt.month
    df_features['is_weekend'] = df_features['day_of_week'].isin([5,6]).astype(int)

# Lag features per junction
if 'Junction' in df_features.columns and 'Vehicles' in df_features.columns:
    df_features = df_features.sort_values(['Junction','DateTime'])
    for lag in [1,2,3]:
        df_features[f'Vehicles_lag_{lag}h'] = df_features.groupby('Junction')['Vehicles'].shift(lag)

# Rolling features: if weather columns exist in merged_df, map them to df_features by DateTime+Junction
weather_cols = ['temp','rhum','wspd','prcp']
if os.path.exists(merged_path):
    merged_df = pd.read_csv(merged_path)
    merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'], dayfirst=True, errors='coerce')
    # merge on DateTime and Junction if both exist
    if 'Junction' in merged_df.columns and 'Junction' in df_features.columns:
        df_features = pd.merge(df_features, merged_df[['DateTime','Junction'] + [c for c in weather_cols if c in merged_df.columns]], on=['DateTime','Junction'], how='left')

# Compute rolling means per junction for weather
for c in weather_cols:
    if c in df_features.columns:
        df_features = df_features.sort_values(['Junction','DateTime'])
        df_features[f'{c}_roll_3h'] = df_features.groupby('Junction')[c].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        df_features[f'{c}_roll_6h'] = df_features.groupby('Junction')[c].transform(lambda x: x.rolling(window=6, min_periods=1).mean())

print('Feature dataframe shape:', df_features.shape)
df_features.head()

## 10. Normalization / Standardization

Scale numeric columns to have zero mean and unit variance (standardization) or scale to [0,1] (normalization) depending on modeling needs. We'll demonstrate StandardScaler from scikit-learn.

In [ ]:
from sklearn.preprocessing import StandardScaler

scale_cols = [c for c in df_features.columns if c not in ['DateTime','Junction','ID'] and df_features[c].dtype in [int,float,'float64','int64']]
# Remove boolean columns
scale_cols = [c for c in scale_cols if df_features[c].nunique()>2]
print('Columns to scale:', scale_cols)

scaler = StandardScaler()
if scale_cols:
    df_features_scaled = df_features.copy()
    df_features_scaled[scale_cols] = scaler.fit_transform(df_features[scale_cols].fillna(0))
else:
    df_features_scaled = df_features.copy()

print('Scaled dataframe sample:')
df_features_scaled.head()

## 11. Feature Importance (Correlation + Random Forest)

We'll show correlation with the target `Vehicles`, and an optional RandomForest feature importance if scikit-learn is available.

In [ ]:
import matplotlib.pyplot as plt

# Correlation with Vehicles
if 'Vehicles' in df_features_scaled.columns:
    corr = df_features_scaled.corr()
    if 'Vehicles' in corr.columns:
        corr_with_target = corr['Vehicles'].sort_values(ascending=False)
        print('Top correlations with Vehicles:')
        print(corr_with_target.head(20))

        plt.figure(figsize=(8,6))
        sns.heatmap(corr.loc[corr_with_target.index[:12], corr_with_target.index[:12]], annot=True, fmt='.2f')
        plt.title('Top feature correlations')
        plt.show()

# RandomForest feature importance (optional)
try:
    from sklearn.ensemble import RandomForestRegressor
    features = [c for c in df_features_scaled.columns if c not in ['DateTime','Junction','ID','Vehicles']]
    features = [c for c in features if df_features_scaled[c].dtype in [int,float,'float64','int64']]
    features = [c for c in features if df_features_scaled[c].nunique()>1]
    print('\nUsing features:', features[:20])
    X = df_features_scaled[features].fillna(0)
    y = df_features_scaled['Vehicles'].fillna(0)
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=1)
    rf.fit(X, y)
    importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
    print('\nTop feature importances:')
    print(importances.head(20))
except Exception as e:
    print('RandomForest failed:', e)
    print('You can install scikit-learn to run this section')

## 12. Export final processed dataset

Save the final processed & feature-engineered dataset for modeling and submission.

In [ ]:
output_path = r'/mnt/data/merged_uber_weather_lag_processed.csv'
df_features_scaled.to_csv(output_path, index=False)
print('Saved processed dataset to:', output_path)

## 13. Final Notes & Next Steps

- This notebook is intentionally verbose and educational (Option A). 
- Next steps: modeling (regression/classification), hyperparameter tuning, cross-validation, and deployment.
- If you'd like, I can also produce a condensed production-ready notebook (Option B) and a PowerPoint summarizing the EDA results.